In [1]:
!pip install salesforce-lavis torchvision
!pip install av tqdm

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━

In [2]:
from lavis.models import load_model_and_preprocess
import torchvision
from torchvision.io import read_video
import torch
import os
from torchvision.transforms import functional as F
import fnmatch
import cv2
from PIL import Image
from tqdm import tqdm
import sys
sys.path.append("AIC2024_Phi_version")

!mkdir ./matrix
!mkdir ./matrix/trainExternal
!mkdir ./matrix/evalExternal

2024-03-05 16:26:06.082502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 16:26:06.082624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 16:26:06.244304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def vid2mat(path: list, name='train'):
    model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device='cuda')
    for vidp in tqdm(path):
        saving_path = f'./matrix/{name}/' + os.path.splitext(os.path.basename(vidp))[0] + '.pt'
        
        # Read video using cv2
        cap = cv2.VideoCapture(vidp)
        frames = []
        features_image_list = []
        while True:
            ret, frame = cap.read()
            # Convert BGR frame to RGB and add to the frames list
            if ret:
                frames.append(F.to_pil_image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

            if len(frames) == 600 or not ret:
                try:
                    # Sample frames every 30 frames
                    sampled_frames = frames[::30]

                    # Preprocess frames using vis_processors
                    images = [vis_processors['eval'](frame).unsqueeze(0) for frame in sampled_frames]

                    # Concatenate the preprocessed images
                    sample = {"image": torch.cat(images, dim=0).to('cuda')}

                    # Extract features using the model
                    features_image_list.append(model.extract_features(sample, mode="image").image_embeds_proj[:, 0, :])

                    frames = []
                except:
                    print(vidp)
                
            if not ret:
                break
        
        # Concat
        features_image = torch.concat(features_image_list, dim=0)
#         print(features_image.shape)
        # Save the features tensor to a file
        torch.save(features_image, saving_path)
        
        del features_image
        del sampled_frames
        del images
        del sample
        del frames

        # Release the video capture object
        cap.release()

In [4]:
def find_mp4_files(folder):
    mp4_files = []
    for root, dirs, files in os.walk(folder):
        for file in fnmatch.filter(files, '*.mp4'):
            mp4_files.append(os.path.abspath(os.path.join(root, file)))
    return mp4_files

train_path = '/kaggle/input/aic24trainingexternal/external/BDD_PC_5K/videos/train'
val_path = '/kaggle/input/aic24trainingexternal/external/BDD_PC_5K/videos/val'

mp4_train = find_mp4_files(train_path)
mp4_val = find_mp4_files(val_path)

In [5]:
vid2mat(mp4_train, 'trainExternal')
vid2mat(mp4_val, 'evalExternal')

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.89G/1.89G [00:10<00:00, 187MB/s]


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

 90%|█████████ | 876/972 [2:17:03<14:52,  9.30s/it]

/kaggle/input/aic24trainingexternal/external/BDD_PC_5K/videos/val/video3345.mp4


 99%|█████████▉| 963/972 [2:30:33<01:22,  9.22s/it]

/kaggle/input/aic24trainingexternal/external/BDD_PC_5K/videos/val/video4783.mp4


100%|██████████| 972/972 [2:31:57<00:00,  9.38s/it]
